In [1]:
from pprint import pprint
from tabulate import tabulate
import pandas as pd
from matplotlib import pyplot as plt

In [3]:

optimizations = ["00", "01","02","03","0s","0z"]
# flag_map = {'--basic-aa':"basic-aa", "tbaa",'--scev-aa':"scev-aa",'--objc-arc-aa':'--globals-aa':"globals-aa","anders-aa",'--cfl-steens-aa':"steens-aa","objc-arc",'--external-aa':"external-aa","scoped-noalias"}
methods =[]
states = ["no","partial","may","must"]
measures = ["time","memory"]

optimizations = ["00", "01","02","03","0s","0z"]

def experiment_retrieval( sum_file ):
    experiment = {}
    for op in optimizations:
        with open("op-"+op+ sum_file) as f:
            summary = {}
            lines = f.readlines()
            for line in lines:
                line = line.strip()
                content= line.split()
                n = len(content)
                if(n>2):
                    aa_method = content[0]
                    if(aa_method not in methods):
                        methods.append(aa_method)
                    aa_method = aa_method[2:]
                    params ={}
                    total = 0
                    for idx,x in enumerate(content[1:]):
                        if(x in states):
                            num = int(content[idx+2])
                            total+= num
                            params[x] = [num,0]
                        elif(x in measures):
                            num = int(content[idx+2])
                            params[x] = num
                    for i in states:
                        params[i][1] = round(100*params[i][0]/total,2)
                    summary[aa_method] = params
                    if(aa_method == 'cfl-steens-aa'):
                        print(total," for ", op)
            experiment[op] = summary
    return experiment
experiment = experiment_retrieval( "/summary.txt" )

pprint(experiment)


488603169  for  00
79815973  for  01
209395854  for  02
583689176  for  03
64928235  for  0s
57541276  for  0z
{'00': {'basic-aa': {'may': [440140470, 90.08],
                     'memory': 6256,
                     'must': [456865, 0.09],
                     'no': [47663481, 9.76],
                     'partial': [342353, 0.07],
                     'time': 160},
        'cfl-anders-aa': {'may': [436025668, 89.24],
                          'memory': 9744,
                          'must': [456865, 0.09],
                          'no': [51778283, 10.6],
                          'partial': [342353, 0.07],
                          'time': 2808},
        'cfl-steens-aa': {'may': [436809320, 89.4],
                          'memory': 8160,
                          'must': [456865, 0.09],
                          'no': [50994631, 10.44],
                          'partial': [342353, 0.07],
                          'time': 177},
        'external-aa': {'may': [440140470, 90.08],
   

In [15]:
table= []
for op in optimizations:
    line = [op]
    for aa in methods:
        alias =aa[2:]
        line.append(experiment[op][alias]["may"][1])
    table.append(line)

print(tabulate(table, headers=methods,tablefmt="fancy_grid"))
##Overall may percentage analysis

╒════╤══════════════╤══════════╤════════════════╤═════════════╤═════════════════╤═══════════════════╤═════════════════╤═══════════════════════╤═══════════════════╕
│    │   --basic-aa │   --tbaa │   --globals-aa │   --scev-aa │   --objc-arc-aa │   --cfl-steens-aa │   --external-aa │   --scoped-noalias-aa │   --cfl-anders-aa │
╞════╪══════════════╪══════════╪════════════════╪═════════════╪═════════════════╪═══════════════════╪═════════════════╪═══════════════════════╪═══════════════════╡
│ 00 │        90.08 │    90.08 │          89.84 │       90.08 │           90.08 │             89.4  │           90.08 │                 90.08 │             89.24 │
├────┼──────────────┼──────────┼────────────────┼─────────────┼─────────────────┼───────────────────┼─────────────────┼───────────────────────┼───────────────────┤
│ 01 │        79.49 │    79.49 │          78.38 │       79.32 │           79.49 │             79.32 │           79.49 │                 79.49 │             79.2  │
├────┼──────────

In [16]:
table_2= []
for op in optimizations:
    line = [op]
    for aa in methods:
        alias =aa[2:]
        no= experiment[op][alias]["no"][0]
        must = experiment[op][alias]["must"][0]
        line.append(1000*must/no)
    table_2.append(line)

print(tabulate(table_2, headers=methods,tablefmt="fancy_grid"))

╒════╤══════════════╤═══════════╤════════════════╤═════════════╤═════════════════╤═══════════════════╤═════════════════╤═══════════════════════╤═══════════════════╕
│    │   --basic-aa │    --tbaa │   --globals-aa │   --scev-aa │   --objc-arc-aa │   --cfl-steens-aa │   --external-aa │   --scoped-noalias-aa │   --cfl-anders-aa │
╞════╪══════════════╪═══════════╪════════════════╪═════════════╪═════════════════╪═══════════════════╪═════════════════╪═══════════════════════╪═══════════════════╡
│ 00 │     9.58522  │  9.58522  │       9.35461  │    9.58528  │        9.58522  │          8.95908  │        9.58522  │              9.58522  │          8.82349  │
├────┼──────────────┼───────────┼────────────────┼─────────────┼─────────────────┼───────────────────┼─────────────────┼───────────────────────┼───────────────────┤
│ 01 │    14.9125   │ 14.9125   │      14.1255   │   14.8584   │       14.9125   │         14.7882   │       14.9125   │             14.9125   │         12.6164   │
├────┼────

In [17]:
table_time= []
for op in optimizations:
    line = [op]
    for aa in methods:
        alias =aa[2:]
        line.append(experiment[op][alias]["time"])
    table_time.append(line)

print(tabulate(table_time, headers=methods,tablefmt="fancy_grid"))
##Execution time analysis in seconds

╒════╤══════════════╤══════════╤════════════════╤═════════════╤═════════════════╤═══════════════════╤═════════════════╤═══════════════════════╤═══════════════════╕
│    │   --basic-aa │   --tbaa │   --globals-aa │   --scev-aa │   --objc-arc-aa │   --cfl-steens-aa │   --external-aa │   --scoped-noalias-aa │   --cfl-anders-aa │
╞════╪══════════════╪══════════╪════════════════╪═════════════╪═════════════════╪═══════════════════╪═════════════════╪═══════════════════════╪═══════════════════╡
│ 00 │          160 │      161 │            186 │         271 │             183 │               177 │             158 │                   163 │              2808 │
├────┼──────────────┼──────────┼────────────────┼─────────────┼─────────────────┼───────────────────┼─────────────────┼───────────────────────┼───────────────────┤
│ 01 │          114 │      116 │            127 │         144 │             120 │               119 │             112 │                   116 │               889 │
├────┼──────────

In [18]:
def sample_dict():
    sample = {}
    for op in optimizations:
        sample[op] = {}
    return sample


def file_detail_extraction(methods):
    file_map = {}
    file_count = 0
    file_details = {}
    for op in optimizations:
        for aa in methods:
            with open("op-"+op+ "/f"+aa+"out.txt") as f:
                lines = f.readlines()
                for line in lines:
                    line = line.strip()
                    content= line.split()
                    if(len(content)>3):
                        if(content[0]!="Final"):
                            file = content[0][22:]
                            if(file not in file_map):
                                file_map[file] = file_count
                                file_details[file_count] = sample_dict()
                                file_count +=1
                            file =  file_map[file]
                            no = int(content[2])
                            may = int(content[4])
                            partial = int(content[6])
                            must = int(content[8])
                            total = no + partial + may+ must
                            dts = [[no,0],[partial,0],[may,0],[must,0],[total,0]]
                            for i in dts:
                                i[1] = round(100*i[0]/total,2)
                            file_details[file][op][aa] = dts
    return file_details


file_details = file_detail_extraction(methods)

In [19]:
considered_aa = ["--cfl-steens-aa","--cfl-anders-aa", "--globals-aa", "--scev-aa"] ##,
variances = {}
for op in optimizations:
    prim = {}
    for key in file_details.keys():
        if '--basic-aa' in file_details[key][op]:
            sec = {}
            may_basic = file_details[key][op]['--basic-aa'][2]
            for aa in considered_aa:
                if aa in file_details[key][op]:
                    may_scev = file_details[key][op][aa][2]
                    if may_basic[0] != may_scev[0]:
                        diff = may_basic[0] - may_scev[0]
                        perc_diff =  round(may_basic[1] - may_scev[1],2)
                        sec[aa] = [diff, perc_diff]
            if len(sec)> 0:
                prim[key] = sec
    variances[op] = prim



In [20]:
percentages = ["0","5","10","20","40"]

def generate_counts(pc):
    counts = [pc+"% <"]
    p = int(pc)
    for op in optimizations:
        c = 0
        for value in variances[op].values():
            for i in value.values():
                if i[1]>p:
                    c+=1
                    break
        counts.append(c)
    return counts


tb_all_file = []
for i in percentages:
    tb_all_file.append(generate_counts(i))
print(tabulate(tb_all_file, headers=optimizations,tablefmt="fancy_grid"))
##The following table represents the application count percentage may reduction done by any AA method in
# comparison to basic-aa

╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│       │   00 │   01 │   02 │   03 │   0s │   0z │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│ 0% <  │ 1374 │  989 │ 1002 │  995 │  993 │  948 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 5% <  │  387 │  247 │  229 │  207 │  255 │  257 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 10% < │  207 │  156 │  135 │  114 │  161 │  147 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 20% < │   91 │   61 │   60 │   53 │   60 │   63 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 40% < │   20 │   16 │   12 │   12 │   17 │   19 │
╘═══════╧══════╧══════╧══════╧══════╧══════╧══════╛


In [21]:
def generate_aa_counts(pc, aa):
    counts = [pc+"% <"]
    p = int(pc)
    for op in optimizations:
        c = 0
        for value in variances[op].values():
            if aa in value:
                if value[aa][1]>p:
                    c+=1
        counts.append(c)
    return counts

for aa in considered_aa:
    tb_all_file = []
    for i in percentages:
        tb_all_file.append(generate_aa_counts(i,aa))
    print("Application Evaluation for:", aa)
    print(tabulate(tb_all_file, headers=optimizations,tablefmt="fancy_grid"))
##The following table represents the application count percentage may reduction done by multiple AA methods  separately in
# comparison to basic-aa


Application Evaluation for: --cfl-steens-aa
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│       │   00 │   01 │   02 │   03 │   0s │   0z │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│ 0% <  │ 1200 │  209 │  227 │  217 │  214 │  219 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 5% <  │  313 │   42 │   41 │   40 │   42 │   37 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 10% < │  174 │   19 │   23 │   23 │   19 │   19 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 20% < │   85 │    8 │    6 │    7 │    6 │    5 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 40% < │   20 │    0 │    0 │    0 │    0 │    0 │
╘═══════╧══════╧══════╧══════╧══════╧══════╧══════╛
Application Evaluation for: --cfl-anders-aa
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│       │   00 │   01 │   02 │   03 │   0s │   0z │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│ 0% <  │ 1260 │  204 │  227 │  221 │  215 │  231 │
├───────┼──────┼──────┼─────

In [22]:
def check_aa_orth(pc, aa1, aa2):
    aa1 = considered_aa[aa1]
    aa2 = considered_aa[aa2]
    counts = [pc+"% <"]
    p = int(pc)
    for op in optimizations:
        a = 0
        b = 0
        a_minus_b = 0
        a_plus_b = 0
        for value in variances[op].values():
            flag_a, flag_b = False, False
            if aa1 in value:
                if value[aa1][1]>p:
                    a+=1
                    flag_a = True
            if aa2 in value:
                if value[aa2][1]>p:
                    b+=1
                    flag_b = True
            if flag_a and flag_b:
                a_minus_b += 1
            if flag_a or flag_b:
                a_plus_b += 1

        c = min(a,b)
        d = max(a,b)
        if c !=0:
            i = round(1-((a_minus_b*a_plus_b))/(a*b),2)
        elif d == 0:
            i = 0
        else:
            i = 1
        counts.append(i)
    return counts

orth_pairs = { "Steens-Anders":(0,1),"Steens-Globals":(0,2),"Steens-Scevs":(0,3),"Globals-Scevs":(2,3),}
for e in orth_pairs.keys():
    tb_orth = []
    print(e)
    for i in percentages:
        tb_orth.append(check_aa_orth(i, orth_pairs[e][0], orth_pairs[e][1]))
    print(tabulate(tb_orth, headers=optimizations, tablefmt="fancy_grid"))
#Following table represent the orthogonality measurements between several pairs of alias analysis methods


Steens-Anders
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│       │   00 │   01 │   02 │   03 │   0s │   0z │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│ 0% <  │    0 │ 0    │ 0    │ 0    │    0 │    0 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 5% <  │    0 │ 0    │ 0.01 │ 0    │    0 │    0 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 10% < │    0 │ 0    │ 0    │ 0    │    0 │    0 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 20% < │    0 │ 0.03 │ 0.05 │ 0.04 │    0 │    0 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 40% < │    0 │ 0    │ 0    │ 0    │    0 │    0 │
╘═══════╧══════╧══════╧══════╧══════╧══════╧══════╛
Steens-Globals
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│       │   00 │   01 │   02 │   03 │   0s │   0z │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│ 0% <  │ 0.25 │ 0.56 │ 0.55 │ 0.56 │ 0.57 │ 0.56 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 5% <  │ 1    │ 0.91 │ 1    │ 1   

In [23]:
methods = []
experiment = experiment_retrieval( "/combined_analysis.txt" )
table= []
for op in optimizations:
    line = [op]
    for aa in methods:
        alias =aa[2:]
        line.append(experiment[op][alias]["may"][1])
    table.append(line)
print(tabulate(table, headers=methods,tablefmt="fancy_grid"))

#Following table represents the overall percentage may reduction by selected pairs of alias analysis methods

╒════╤═════════════════════════════╤══════════════════════════╤════════════════════════════════╕
│    │   --cfl-steens-aa+--scev-aa │   --globals-aa+--scev-aa │   --cfl-steens-aa+--globals-aa │
╞════╪═════════════════════════════╪══════════════════════════╪════════════════════════════════╡
│ 00 │                       89.4  │                    89.84 │                          89.16 │
├────┼─────────────────────────────┼──────────────────────────┼────────────────────────────────┤
│ 01 │                       79.15 │                    78.2  │                          78.21 │
├────┼─────────────────────────────┼──────────────────────────┼────────────────────────────────┤
│ 02 │                       51.61 │                    51.27 │                          51.58 │
├────┼─────────────────────────────┼──────────────────────────┼────────────────────────────────┤
│ 03 │                       32.53 │                    32.5  │                          32.58 │
├────┼────────────────────────

In [24]:
extended_file_details = file_detail_extraction(methods)

In [27]:
extended_file_details

{0: {'00': {'--cfl-steens-aa+--scev-aa': [[6945, 45.73],
    [14, 0.09],
    [8209, 54.06],
    [18, 0.12],
    [15186, 100.0]],
   '--globals-aa+--scev-aa': [[7132, 46.96],
    [14, 0.09],
    [8022, 52.82],
    [18, 0.12],
    [15186, 100.0]],
   '--cfl-steens-aa+--globals-aa': [[7187, 47.33],
    [14, 0.09],
    [7967, 52.46],
    [18, 0.12],
    [15186, 100.0]]},
  '01': {'--cfl-steens-aa+--scev-aa': [[16103, 66.7],
    [19, 0.08],
    [7953, 32.94],
    [66, 0.27],
    [24141, 100.0]],
   '--globals-aa+--scev-aa': [[16882, 69.93],
    [19, 0.08],
    [7174, 29.72],
    [66, 0.27],
    [24141, 100.0]],
   '--cfl-steens-aa+--globals-aa': [[16882, 69.93],
    [19, 0.08],
    [7174, 29.72],
    [66, 0.27],
    [24141, 100.0]]},
  '02': {'--cfl-steens-aa+--scev-aa': [[15803, 66.62],
    [19, 0.08],
    [7832, 33.02],
    [66, 0.28],
    [23720, 100.0]],
   '--globals-aa+--scev-aa': [[16582, 69.91],
    [19, 0.08],
    [7053, 29.73],
    [66, 0.28],
    [23720, 100.0]],
   '--cfl-steens

In [29]:
re_considered_aa = ["--cfl-steens-aa+--scev-aa","--globals-aa+--scev-aa", "--cfl-steens-aa+--globals-aa"] ##,
variances = {}
for op in optimizations:
    prim = {}
    for key in extended_file_details.keys():
        if '--basic-aa' in file_details[key][op]:
            sec = {}
            may_basic = file_details[key][op]['--basic-aa'][2]
            for aa in re_considered_aa:
                if aa in extended_file_details[key][op]:
                    may_scev = extended_file_details[key][op][aa][2]
                    if may_basic[0] != may_scev[0]:
                        diff = may_basic[0] - may_scev[0]
                        perc_diff =  round(may_basic[1] - may_scev[1],2)
                        sec[aa] = [diff, perc_diff]
            if len(sec)> 0:
                prim[key] = sec
    variances[op] = prim


In [30]:
variances

{'00': {0: {'--cfl-steens-aa+--scev-aa': [55, 0.36],
   '--globals-aa+--scev-aa': [242, 1.6],
   '--cfl-steens-aa+--globals-aa': [297, 1.96]},
  1: {'--cfl-steens-aa+--scev-aa': [977, 7.13],
   '--cfl-steens-aa+--globals-aa': [977, 7.13]},
  4: {'--cfl-steens-aa+--scev-aa': [893, 22.7],
   '--cfl-steens-aa+--globals-aa': [893, 22.7]},
  5: {'--cfl-steens-aa+--scev-aa': [103, 0.1],
   '--cfl-steens-aa+--globals-aa': [103, 0.1]},
  6: {'--cfl-steens-aa+--scev-aa': [12, 0.02],
   '--cfl-steens-aa+--globals-aa': [12, 0.02]},
  8: {'--cfl-steens-aa+--scev-aa': [784, 8.42],
   '--cfl-steens-aa+--globals-aa': [784, 8.42]},
  9: {'--cfl-steens-aa+--scev-aa': [58, 0.13],
   '--cfl-steens-aa+--globals-aa': [58, 0.13]},
  13: {'--cfl-steens-aa+--scev-aa': [20, 0.61],
   '--cfl-steens-aa+--globals-aa': [20, 0.61]},
  14: {'--cfl-steens-aa+--scev-aa': [734, 0.59],
   '--cfl-steens-aa+--globals-aa': [734, 0.59]},
  16: {'--cfl-steens-aa+--scev-aa': [8, 0.02],
   '--cfl-steens-aa+--globals-aa': [8, 0

In [31]:
def generate_aa_counts(pc, aa):
    counts = [pc+"% <"]
    p = int(pc)
    for op in optimizations:
        c = 0
        for value in variances[op].values():
            if aa in value:
                if value[aa][1]>p:
                    c+=1
        counts.append(c)
    return counts

for aa in re_considered_aa:
    tb_all_file = []
    for i in percentages:
        tb_all_file.append(generate_aa_counts(i,aa))
    print("Application Evaluation for:", aa)
    print(tabulate(tb_all_file, headers=optimizations,tablefmt="fancy_grid"))

Application Evaluation for: --cfl-steens-aa+--scev-aa
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│       │   00 │   01 │   02 │   03 │   0s │   0z │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│ 0% <  │ 1208 │  800 │  841 │  838 │  811 │  756 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 5% <  │  315 │  118 │  110 │  105 │  123 │  117 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 10% < │  175 │   68 │   62 │   49 │   69 │   57 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 20% < │   85 │   11 │   15 │   15 │   10 │   10 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│ 40% < │   20 │    0 │    0 │    0 │    0 │    1 │
╘═══════╧══════╧══════╧══════╧══════╧══════╧══════╛
Application Evaluation for: --globals-aa+--scev-aa
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│       │   00 │   01 │   02 │   03 │   0s │   0z │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│ 0% <  │  374 │  945 │  961 │  956 │  942 │  896 │
├───────┼──